# Cell Tracking and Lineage Construction in Live-Cell Imaging Data
---


### Global Imports

In [5]:
import os
import datetime
import errno
import argparse

import numpy as np

import deepcell

### Load the Training Data

In [6]:
# Download the data (saves to ~/.keras/datasets)
filename = '3T3_NIH.trks'
(X_train, y_train), (X_test, y_test) = deepcell.datasets.tracked.nih_3t3.load_tracked_data(filename)

print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

X.shape: (188, 30, 154, 182, 1)
y.shape: (188, 30, 154, 182, 1)


### Set up filepath constants

In [7]:
# The path to the data file is currently required for `train_model_()` functions

# Change DATA_DIR if you are not using `deepcell.datasets`
DATA_DIR = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
DATA_DIR = "/data/npz_data/cells/HEK293/generic/movie/"                # USE LOCAL DATA INSTEAD

# DATA_FILE should be a trks file (contains 2 np arrays and a lineage dictionary)
DATA_FILE = os.path.join(DATA_DIR, filename)
DATA_FILE = os.path.join(DATA_DIR, '3T3_HeLa_HEK_corrected.trks')      # USE LOCAL DATA INSTEAD

# confirm the data file is available
assert os.path.isfile(DATA_FILE)

In [8]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = os.path.relpath(os.path.dirname(DATA_FILE), DATA_DIR)

ROOT_DIR = '/data'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

### Set up training parameters

In [9]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

tracking_model_name = 'tracking_model'

n_epoch = 5  # Number of training epochs
test_size = .10  # % of data saved as test

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Tracking training settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 5
neighborhood_scale_size=30
batch_size = 128  

in_shape = (32, 32, 1) # Should this be calculated or hardcoded?
seed = 100 # To be removed

### Training the Model

#### Instantiate the tracking model

In [10]:
from deepcell import model_zoo

tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

#### Option 1: Train a new tracking model

In [ ]:
from deepcell.training import train_model_siamese_daughter

tracking_model = train_model_siamese_daughter(
    model=tracking_model,
    dataset=DATA_FILE,  # full path to trks file
    model_name=tracking_model_name,
    optimizer=optimizer,
    batch_size=batch_size,
    min_track_length=min_track_length,
    features=features,
    neighborhood_scale_size=neighborhood_scale_size,
    n_epoch=n_epoch,
    model_dir=MODEL_DIR,
    lr_sched=lr_sched,
    rotation_range=180,
    flip=True,
    shear=False,
    class_weight=None,
    seed = seed)

#### Option 2: Load an existing tracking model

In [16]:
# Import the tracking model
MODEL_DIR = '/data/models/'
PREFIX = 'cells/HEK293/generic/'

# Re-instantiate the model and load weights
siamese_weights_file = '2019-01-24_3T3_HeLa_HEK_corrected_[a,d,n,r]_neighs=30_epochs=5_seed=360_trks_0.h5'
siamese_weights_file = os.path.join(MODEL_DIR, PREFIX, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### Verify Model Accuracy with Confusion Matrix

In [12]:
# Using DATA_FILE from above to extract Test Data 
# Change if you are not using `deepcell.datasets`

In [13]:
import deepcell.image_generators as generators
from deepcell.utils.data_utils import get_data

train_dict, test_dict = get_data(DATA_FILE, mode='siamese_daughters', seed=seed)

datagen_test = generators.SiameseDataGenerator(
        rotation_range=0,  # randomly rotate images by 0 to rotation_range degrees
        shear_range=0,     # randomly shear images in the range (radians , -shear_range to shear_range)
        horizontal_flip=0, # randomly flip images
        vertical_flip=0)   # randomly flip images

test_iterator = generators.SiameseIterator(test_dict,
                                           datagen_test,
                                           neighborhood_scale_size=neighborhood_scale_size,
                                           min_track_length=min_track_length,
                                           features=features)

/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

In [25]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []
for i in range(1,1001):
    if i % 100 == 0:
        print(".", end="")
    lst, y_true = next(test_iterator)
    y_true = list(map(np.argmax, y_true))
    y_pred = list(map(np.argmax, tracking_model.predict(lst)))
    Y.extend(y_true)
    Y_pred.extend(y_pred)

confusion_matrix(Y, Y_pred)

..........

array([[10753,     2,     1],
       [    4, 10511,     3],
       [   98,  3216,  7346]])

In [27]:
test_acc = sum(np.array(Y) == np.array(Y_pred)) / len(Y)
print('Accuracy across all three classes: ', test_acc)

Accuracy across all three classes:  0.8959103150247385


### Track Multiple Movies and Generate Track Files

#### Run the model

In [14]:
# Using DATA_FILE from above for example Test Data 
# Change if you are not using `deepcell.datasets`

In [29]:
# Normalize raw images if needed
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

for batch in range(test_dict['X'].shape[0]):
    for frame in range(test_dict['X'].shape[1]):
        test_dict['X'][batch, frame, :, :, 0] = image_norm(test_dict['X'][batch, frame, :, :, 0]) 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [54]:
# The tracking model is used in concert with other processes to track cells
# Import the neccesary tracking functionality
import deepcell.tracking

# Define where cell tracks will be saved
TRACK_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'track_data', PREFIX))
TRACK_FILE_NAME = 'batch_'

# create directories if they do not exist
try:
    os.makedirs(TRACK_DIR)
except OSError as exc:  # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise

# Depending on the number of batches you may not want to track everything at once
#num_batches = test_dict['X'].shape[0]
num_batches = 2

# Generate a cell track for each batch
#for batch in range(test_dict['X'].shape[0]):
for batch in range(num_batches):
    trial = deepcell.tracking.cell_tracker(test_dict['X'][batch], test_dict['y'][batch],
                         tracking_model,
                         max_distance=200,
                         track_length=5, division=0.5, birth=0.9, death=0.9,
                         neighborhood_scale_size=30,
                         features=features)
    trial._track_cells()
    file_name = TRACK_FILE_NAME + str(batch).zfill(2) + '.trk'
    file_path = os.path.join(TRACK_DIR, file_name)
    trial.dump(file_path)

/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabl

Tracking frame 1
Tracking frame 2
Tracking frame 3
Tracking frame 4
Tracking frame 5
Tracking frame 6
Tracking frame 7
New track
New track
Tracking frame 8
Tracking frame 9
Tracking frame 10
Tracking frame 11
Tracking frame 12
Tracking frame 13
Tracking frame 14
Tracking frame 15
Tracking frame 16
Tracking frame 17
Tracking frame 18
Tracking frame 19
Tracking frame 20
Tracking frame 21
New track
Division detected
New track
Division detected
Tracking frame 22
Tracking frame 23
Tracking frame 24
Tracking frame 25
Tracking frame 26
Tracking frame 27
Tracking frame 28
Tracking frame 29
Tracking frame 30
Tracking frame 31
Tracking frame 32
Tracking frame 33
Tracking frame 34
Tracking frame 35
Tracking frame 36
Tracking frame 37
Tracking frame 38
Tracking frame 39
Tracking frame 1
Tracking frame 2
Tracking frame 3
Tracking frame 4
Tracking frame 5
Tracking frame 6
New track
Tracking frame 7
Tracking frame 8
Tracking frame 9
Tracking frame 10
Tracking frame 11
Tracking frame 12
Tracking frame

#### Bundle individual track files (each batch) into one .trks file for review

In [64]:
import importlib
importlib.reload(deepcell.utils.data_utils)

from deepcell.utils.data_utils import trk_folder_to_trks
from deepcell.utils.data_utils import load_trks


# Define a name for the trks file
SET_NAME = 'all_batches.trks'

# Compile trk files into one trks file
trk_folder_to_trks(TRACK_DIR,SET_NAME)

FILE_PATH = os.path.join(TRACK_DIR, SET_NAME)
trks = load_trks(FILE_PATH)

### Review the results

In [ ]:
# Load the file we created above to review
lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

#### Tracked Movies

In [69]:
# View tracked results of each batch as a video
# NB: This does not render well on GitHub
from IPython.display import HTML
from deepcell.utils.plot_utils import get_js_video

# Change this value to look at other batches of data
batch = 0

# Raw
HTML(get_js_video(raw, batch=batch, cmap='gray'))

In [70]:
# Tracked
HTML(get_js_video(tracked, batch=batch, cmap='jet'))

#### Save the Raw and Tracked Output as Images

In [74]:
import matplotlib.pyplot as plt

# Define where images (movies) will be saved
MOVIE_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'tracked_movies', PREFIX))

# create directories if they do not exist
try:
    os.makedirs(MOVIE_DIR)
except OSError as exc:  # Guard against race condition
    if exc.errno != errno.EEXIST:
        raise
        
channel = 0 # These images should only have one channel
for i in range(raw.shape[1]):
    name_raw = os.path.join(MOVIE_DIR,'raw_frame_{:02}_.png'.format(i))
    name_tracked = os.path.join(MOVIE_DIR,'tracked_frame_{:02}_.png'.format(i))
    plt.imsave(name_raw, raw[batch, i, :, :, channel], cmap='gray')
    plt.imsave(name_tracked, tracked[batch,i, :, :, channel], cmap='cubehelix', vmin=0, vmax=30)